In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Метрики
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
# df = pd.read_csv('../topic_modeling_task_sample_trainPart.csv')
# df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
# df.head()

In [57]:
df = pd.read_csv('../ready.csv').dropna()

In [58]:
# Есть ли пропуски:
print(df.isnull().sum())
# Есть ли дубликаты
print(df.duplicated().any())
print(len(df))

Unnamed: 0                         0
ucid                               0
message                            0
ACTION_ITEM_RESULT_PRODUCT_NAME    0
dtype: int64
False
42620


# Очистка текста

In [59]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df.ACTION_ITEM_RESULT_PRODUCT_NAME = le.fit_transform(df.ACTION_ITEM_RESULT_PRODUCT_NAME)

In [60]:
X, y = df.message, df.ACTION_ITEM_RESULT_PRODUCT_NAME

In [61]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X.values.reshape(-1, 1), y)

## TfidfVectorizer

In [67]:
tfidfconverter = TfidfVectorizer(max_features=3000, min_df=5, max_df=0.7, stop_words=stopwords.words('russian'))
X_resampled_vect = tfidfconverter.fit_transform(pd.DataFrame(X_resampled)[0]).toarray()
X_vect = tfidfconverter.transform(X)

In [68]:
X_train, _, y_train, _ = train_test_split(X_resampled_vect, y_resampled, test_size=0.3, random_state=42)
_, X_test, _, y_test = train_test_split(X_vect, y, test_size=0.3, random_state=42)


In [69]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [70]:
y_pred = rf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[4463  401  762  143]
 [ 173 2928  501  100]
 [  83  124 1796   59]
 [  19   23   61 1150]]
              precision    recall  f1-score   support

           0       0.94      0.77      0.85      5769
           1       0.84      0.79      0.82      3702
           2       0.58      0.87      0.69      2062
           3       0.79      0.92      0.85      1253

    accuracy                           0.81     12786
   macro avg       0.79      0.84      0.80     12786
weighted avg       0.84      0.81      0.81     12786

0.8084623807289223


In [72]:
import joblib


joblib.dump(tfidfconverter, '../models/tfidfconverter.pkl')
joblib.dump(rf, '../models/randomforest.pkl')

['../models/randomforest.pkl']